今天在github上找到一个股价预测的项目， 感觉该项目对很多会计金融领域的同学很有帮助。

```
项目:Stock-Price-Predictions
作者:Prince Kumar, Simon Jones, Amratansh Sharma, Lorena Bustamante
链接: https://github.com/amratansh/Stock-Price-Predictions
译者:大邓
```

## 本文目录
- 摘要
- 分析发现
- 项目EasyInvest的PPT
- 文本分析情感库安装（Loughran&MCdonald）
- 报告文本分析实战

### 摘要
作者分析了Morningstar网站上15家最大的零售及科技类公司的分析师报告，并提取了相关的文本数据。然后使用两种情感分析方法得到两种情感值，第一种是使用通用的情感分析库（VADER）分析广义上的情感值（形容词情感，如喜怒哀乐），第二种是情感值使用Loughran&MCdonald金融情感词典得到专业领域情感值；  同时还使用S&P Capital IQ和 Yahool Finance 作为原始数据源。最后使用stata做了分析并展示了分析结果。

### 分析发现
在控制其他指标情况下，结果发现:
1、 下一交易日(T1)股价预测  - 使用分析师报告标题和第一段文本的情感得分，报告当天的情感值每增加10%，下一交易日股价上涨0.5%。
2、 下周(T5)股价预测       - 使用分析师报告整体文本的情感得分，报告当天的情感值每增加10%，下周股价上涨0.2%。
3、 下月(T30)股价预测      - 我们的模型没能做出分析师报告当天情感值对30田后的股价变动有预测效应


本次分享的内容包括
- PPT
- 代码


## 一、EasyInvest PPT
这个项目发布于2018年12月6日，感觉挺新的，也是大邓在网上同时找到计算思路、数据和代码的项目。项目名叫做EasyInvests，大家先拿出点时间先看一下该项目的PPT，代码会放到PPT之后（都是大邓调试后可运行的哦）。

### ppt

sss


sss

## 二、报告文本分析实战
预测股价用到了 S&P Capital IQ 、 Yahool Finance、 Morningstar三个数据源，其中Morningstar主要是用来抽取分析师报告中的情感值，是唯一的非结构化数据。

项目作者对S&P Capital IQ 、 Yahool Finance一笔带过，主要讲解的是 

**如何从Morningstar分析师报告中提取情感值的技术和方法**

我相信这也是各位最感兴趣的地方。

#### 文本情感分析库安装
本文用到的文本情感分析库:
- pysentiment    基于Loughran&MCdonald金融情感词典制作的情感分析库
- vaderSentiment nltk内的通用情感分析模块

**vaderSentiment安装方法**

In [ ]:
!pip3 install vaderSentiment

**pysentiment安装方法**

In [ ]:
!pip3 install pysentiment

我安装pysentiment遇到问题（估计你们也会遇到），解决办法:
1. 下载本文的项目(文章末尾有下载链接获取方式)
2. 将项目文件夹压缩包解压，解压到桌面。
3. cmd打开命令行（不懂的百度）
4. 命令行输入  ``cd desktop``,按 Enter回车键
5. 命令行输入  ``cd Stock-Price-Predictions``,按 Enter回车键
6. 命令行输入  ``cd pysentiment``,按 Enter回车键
7. 命令行输入  ``python3 setup.py install``,按 Enter回车键。 有的同学这里如果有问题，可以将python3换成python

经过上面的操作，pysentiment库也就安装好了。

### 2.1 导入数据

In [4]:
import pandas as pd
from IPython.display import clear_output

#windows，注意与mac路径写法不同
#df = pandas.read_excel("data\\Analyst_reports1.xls", sheet_name="Sheet1")

#mac
df = pd.read_excel("data/Analyst_reports1.xls", sheet_name="Sheet1")
df.head()

# df1

用 ``分析师报告标题、第一自然段、最后自然段`` 三个字段合并新建一个字段 ``Combined``

更改日期格式

In [3]:
df["Combined"]=df["Headline"]+df["First Paragraph"]+df["Last Paragraph"]

df["Date"]=df["Date"].apply(pd.to_datetime)

df.head()

# df2

### 2.2 定义情感函数
pandas的dataframe提供了很强大的列数据操作，只要我们定义好文本情感计算的函数，就可以对excel中的某字段进行批量操作。

这里用到

```python
df['colname'].apply(func)
```

**参数解读:**
- colname 字段名(列名)
- func  对colname字段进行计算，比如计算Combined列的情感得分

昨天分享的pysentiment库计算结果能得到 ``Positive、Negative、Polarity、Subjectivity``


**指标说明**

```
Positive    正面词词频数
Negative    负面词词频数
Polarity =(Pos-Neg)/(Pos+Neg)
Subjectivity =(Pos+Neg)/count(*)
```

In [4]:
from pysentiment import LM
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

lm = LM()
sia = SentimentIntensityAnalyzer()


def finance_score(text):
    """
    输入一段文本，返回LM金融情感得分（Polarity得分）
    该函数使用的是pysentiment库，该词库基于Loughran&MCdonald开发的金融情感词库
    """
    words = lm.tokenize(text)
    scores = lm.get_score(words)
    return scores["Polarity"]

def general_score(text):
    """
    输入一段文本，返回通用情感得分（compound得分）
    该函数使用的是vaderSentiment库
    """
    scores =sia.polarity_scores(text)
    return scores['compound']

### 2.3 提取情感得分

In [ ]:
df['Finance Headline'] = df['Headline'].apply(finance_score)
df['Finance First Paragraph'] = df['First Paragraph'].apply(finance_score)
df['Finance Last Paragraph'] = df['Last Paragraph'].apply(finance_score)
df['Finance Combined'] = df['Combined'].apply(finance_score)

df['General Headline'] = df['Headline'].apply(general_score)
df['General First Paragraph'] = df['First Paragraph'].apply(general_score)
df['General Last Paragraph'] = df['Last Paragraph'].apply(general_score)
df['General Combined'] = df['Combined'].apply(general_score)

selected_fields = ['Company', 'Date', 'Finance Headline','Finance First Paragraph',
                  'Finance Last Paragraph','Finance Combined','General Headline',
                  'General First Paragraph','General Last Paragraph','General Combined']
df[selected_fields].head()

# df3
### 2.4 保存数据

In [26]:
df[selected_fields].to_csv('output/Sentiments_Final_really.csv')